# 컨텍스트 검색 

- 컨텍스트 검색의 핵심 아이디어는 각 청크에 관련 컨텍스트를 추가해 필요한 청크를 더 쉽게 검색할 수 있게 하는 것이다. 
  - 간단한 방법은 태그나 키워드 같은 메타데이터로 청크를 보강하는 하는 것이다. 
  - 전자 상거래의 경우, 상품에 설명과 리뷰를 함께 저장할 수 있다. 
  - 각 청크에 질의들을 추가할 수 있다. 고객 지원의 경우, 각 문서에 관련 질의들을 추가할 수 있다. 

In [ ]:
"""
컨텍스트 검색 (Contextual Retrieval) 시스템

핵심 아이디어: 각 청크에 관련 컨텍스트(메타데이터, 키워드, 질의)를 추가하여
검색 품질을 향상시킴
"""

from dataclasses import dataclass, field
from typing import Optional
import numpy as np
from collections import Counter
import re


@dataclass
class EnrichedChunk:
    """컨텍스트가 보강된 청크"""
    id: str
    content: str  # 원본 텍스트
    
    # 메타데이터
    tags: list[str] = field(default_factory=list)
    keywords: list[str] = field(default_factory=list)
    category: Optional[str] = None
    
    # 관련 질의들 (이 청크가 답변할 수 있는 질문들)
    related_queries: list[str] = field(default_factory=list)
    
    # 추가 컨텍스트
    summary: Optional[str] = None
    parent_context: Optional[str] = None  # 상위 문서의 컨텍스트
    
    def get_searchable_text(self) -> str:
        """검색에 사용될 확장된 텍스트 반환"""
        parts = [self.content]
        
        if self.tags:
            parts.append(f"태그: {', '.join(self.tags)}")
        if self.keywords:
            parts.append(f"키워드: {', '.join(self.keywords)}")
        if self.related_queries:
            parts.append(f"관련 질의: {' | '.join(self.related_queries)}")
        if self.summary:
            parts.append(f"요약: {self.summary}")
        if self.parent_context:
            parts.append(f"문맥: {self.parent_context}")
            
        return "\n".join(parts)


# =============================================================================
# 예시 1: 전자상거래 - 상품 정보 + 리뷰
# =============================================================================

@dataclass
class ProductChunk(EnrichedChunk):
    """전자상거래용 상품 청크"""
    product_name: Optional[str] = None
    price: Optional[float] = None
    reviews: list[str] = field(default_factory=list)
    rating: Optional[float] = None
    
    def get_searchable_text(self) -> str:
        base = super().get_searchable_text()
        parts = [base]
        
        if self.reviews:
            parts.append(f"리뷰: {' '.join(self.reviews)}")
        if self.rating:
            parts.append(f"평점: {self.rating}")
            
        return "\n".join(parts)


def create_product_chunk(
    product_id: str,
    name: str,
    description: str,
    reviews: list[str],
    price: float,
    rating: float
) -> ProductChunk:
    """상품 정보로부터 보강된 청크 생성"""
    
    # 키워드 자동 추출 (간단한 방식)
    all_text = f"{name} {description} {' '.join(reviews)}"
    keywords = extract_keywords(all_text)
    
    # 관련 질의 자동 생성
    related_queries = [
        f"{name} 가격",
        f"{name} 리뷰",
        f"{name} 추천",
        f"{name} 장단점",
    ]
    
    # 리뷰에서 태그 추출
    tags = extract_sentiment_tags(reviews)
    
    return ProductChunk(
        id=product_id,
        content=description,
        product_name=name,
        price=price,
        reviews=reviews,
        rating=rating,
        keywords=keywords,
        related_queries=related_queries,
        tags=tags,
        summary=f"{name} - {rating}점, {len(reviews)}개 리뷰"
    )


# =============================================================================
# 예시 2: 고객 지원 - FAQ 문서 + 관련 질의
# =============================================================================

@dataclass 
class SupportChunk(EnrichedChunk):
    """고객 지원용 문서 청크"""
    document_title: Optional[str] = None
    solution_steps: list[str] = field(default_factory=list)
    
    def get_searchable_text(self) -> str:
        base = super().get_searchable_text()
        parts = [base]
        
        if self.solution_steps:
            parts.append(f"해결 방법: {' -> '.join(self.solution_steps)}")
            
        return "\n".join(parts)


def create_support_chunk(
    doc_id: str,
    title: str,
    content: str,
    user_queries: list[str],  # 실제 고객들이 했던 질문들
    solution_steps: list[str] = None
) -> SupportChunk:
    """고객 지원 문서로부터 보강된 청크 생성"""
    
    keywords = extract_keywords(f"{title} {content}")
    
    # 질의 변형 생성 (다양한 표현 방식)
    expanded_queries = expand_queries(user_queries)
    
    return SupportChunk(
        id=doc_id,
        content=content,
        document_title=title,
        keywords=keywords,
        related_queries=expanded_queries,
        solution_steps=solution_steps or [],
        tags=["고객지원", "FAQ"],
        summary=title
    )


# =============================================================================
# 유틸리티 함수들
# =============================================================================

def extract_keywords(text: str, top_n: int = 10) -> list[str]:
    """텍스트에서 키워드 추출 (간단한 빈도 기반)"""
    # 불용어 (실제로는 더 많이 필요)
    stopwords = {'이', '그', '저', '것', '수', '등', '및', '의', '를', '을', '에', '가', 
                 '은', '는', '으로', '로', '와', '과', '에서', '하는', '하다', '있다',
                 'the', 'a', 'an', 'is', 'are', 'was', 'were', 'be', 'been', 'and', 'or'}
    
    # 단어 추출 (한글 + 영어)
    words = re.findall(r'[가-힣]+|[a-zA-Z]+', text.lower())
    
    # 불용어 제거 및 짧은 단어 제거
    words = [w for w in words if w not in stopwords and len(w) > 1]
    
    # 빈도 계산
    counter = Counter(words)
    
    return [word for word, _ in counter.most_common(top_n)]


def extract_sentiment_tags(reviews: list[str]) -> list[str]:
    """리뷰에서 감성 태그 추출"""
    tags = []
    all_reviews = " ".join(reviews).lower()
    
    positive_words = ['좋', '만족', '추천', '최고', '훌륭', '편리', '빠른', 'good', 'great', 'excellent']
    negative_words = ['나쁜', '불만', '별로', '실망', '느린', 'bad', 'poor', 'terrible']
    
    for word in positive_words:
        if word in all_reviews:
            tags.append("긍정적")
            break
            
    for word in negative_words:
        if word in all_reviews:
            tags.append("개선필요")
            break
            
    return tags if tags else ["중립"]


def expand_queries(queries: list[str]) -> list[str]:
    """질의를 다양한 형태로 확장"""
    expanded = list(queries)  # 원본 유지
    
    for query in queries:
        # 질문 형태 변형
        if "어떻게" not in query:
            expanded.append(f"{query} 어떻게")
        if "방법" not in query:
            expanded.append(f"{query} 방법")
        if "?" not in query:
            expanded.append(f"{query}?")
            
    return list(set(expanded))  # 중복 제거


# =============================================================================
# 검색 시스템
# =============================================================================

class ContextualRetriever:
    """컨텍스트 기반 검색기"""
    
    def __init__(self):
        self.chunks: list[EnrichedChunk] = []
        
    def add_chunk(self, chunk: EnrichedChunk):
        """청크 추가"""
        self.chunks.append(chunk)
        
    def add_chunks(self, chunks: list[EnrichedChunk]):
        """여러 청크 추가"""
        self.chunks.extend(chunks)
        
    def search(self, query: str, top_k: int = 5) -> list[tuple[EnrichedChunk, float]]:
        """
        검색 수행 (간단한 키워드 매칭 기반)
        실제 구현에서는 임베딩 기반 검색 사용 권장
        """
        query_keywords = set(extract_keywords(query, top_n=20))
        results = []
        
        for chunk in self.chunks:
            # 확장된 검색 텍스트 사용
            searchable_text = chunk.get_searchable_text().lower()
            chunk_keywords = set(extract_keywords(searchable_text, top_n=50))
            
            # 관련 질의와의 매칭 점수 (보너스)
            query_match_bonus = 0
            for related_query in chunk.related_queries:
                if any(kw in related_query.lower() for kw in query_keywords):
                    query_match_bonus += 0.2
                    
            # 키워드 겹침 점수
            overlap = len(query_keywords & chunk_keywords)
            score = overlap / max(len(query_keywords), 1) + query_match_bonus
            
            results.append((chunk, score))
            
        # 점수순 정렬
        results.sort(key=lambda x: x[1], reverse=True)
        
        return results[:top_k]


# =============================================================================
# 사용 예시
# =============================================================================

def demo_ecommerce():
    """전자상거래 예시"""
    print("=" * 60)
    print("📦 전자상거래 컨텍스트 검색 예시")
    print("=" * 60)
    
    # 상품 청크 생성
    products = [
        create_product_chunk(
            product_id="P001",
            name="무선 블루투스 이어폰",
            description="고음질 노이즈 캔슬링 기능이 탑재된 완전 무선 이어폰입니다. 배터리 수명 8시간, 충전 케이스 포함.",
            reviews=[
                "음질이 정말 좋아요! 노이즈 캔슬링 최고입니다.",
                "배터리가 오래가서 만족해요.",
                "가성비 추천합니다."
            ],
            price=89000,
            rating=4.5
        ),
        create_product_chunk(
            product_id="P002", 
            name="기계식 키보드",
            description="청축 스위치 기계식 키보드. RGB 백라이트, 텐키리스 87키 레이아웃.",
            reviews=[
                "타건감이 좋습니다. 청축 특유의 클릭음이 시원해요.",
                "RGB 조명이 예쁘네요.",
                "사무실에서 쓰기엔 소음이 좀 있어요."
            ],
            price=79000,
            rating=4.2
        ),
    ]
    
    # 검색기 생성 및 청크 추가
    retriever = ContextualRetriever()
    retriever.add_chunks(products)
    
    # 검색 테스트
    queries = [
        "노이즈 캔슬링 이어폰 추천",
        "타건감 좋은 키보드",
        "배터리 오래가는 이어폰"
    ]
    
    for query in queries:
        print(f"\n🔍 검색: '{query}'")
        results = retriever.search(query, top_k=2)
        
        for chunk, score in results:
            print(f"  → {chunk.product_name} (점수: {score:.2f})")
            print(f"     키워드: {', '.join(chunk.keywords[:5])}")
            print(f"     평점: {chunk.rating}")


def demo_support():
    """고객 지원 예시"""
    print("\n" + "=" * 60)
    print("🎧 고객 지원 컨텍스트 검색 예시")
    print("=" * 60)
    
    # 지원 문서 청크 생성
    docs = [
        create_support_chunk(
            doc_id="D001",
            title="비밀번호 재설정 방법",
            content="비밀번호를 잊어버린 경우, 로그인 페이지에서 '비밀번호 찾기'를 클릭하여 재설정할 수 있습니다.",
            user_queries=[
                "비밀번호를 잊어버렸어요",
                "로그인이 안돼요",
                "비밀번호 변경하고 싶어요"
            ],
            solution_steps=["로그인 페이지 접속", "'비밀번호 찾기' 클릭", "이메일 입력", "인증 메일 확인", "새 비밀번호 설정"]
        ),
        create_support_chunk(
            doc_id="D002",
            title="환불 정책 안내",
            content="상품 수령 후 7일 이내에 환불 신청이 가능합니다. 단, 사용 흔적이 있는 경우 환불이 제한될 수 있습니다.",
            user_queries=[
                "환불하고 싶어요",
                "반품 어떻게 해요",
                "교환 가능한가요"
            ],
            solution_steps=["마이페이지 접속", "주문 내역 확인", "환불 신청 클릭", "사유 선택", "신청 완료"]
        ),
    ]
    
    retriever = ContextualRetriever()
    retriever.add_chunks(docs)
    
    # 검색 테스트 (다양한 표현 방식)
    queries = [
        "비밀번호가 기억 안나요",
        "물건 반품하려면",
        "로그인 문제"
    ]
    
    for query in queries:
        print(f"\n🔍 검색: '{query}'")
        results = retriever.search(query, top_k=1)
        
        for chunk, score in results:
            if isinstance(chunk, SupportChunk):
                print(f"  → {chunk.document_title} (점수: {score:.2f})")
                print(f"     관련 질의: {chunk.related_queries[:3]}")
                if chunk.solution_steps:
                    print(f"     해결 단계: {' → '.join(chunk.solution_steps[:3])}...")


def demo_chunk_enrichment():
    """청크 보강 과정 시각화"""
    print("\n" + "=" * 60)
    print("📝 청크 보강 과정")
    print("=" * 60)
    
    # 원본 텍스트
    original = "고음질 노이즈 캔슬링 기능이 탑재된 완전 무선 이어폰입니다."
    
    print(f"\n[원본 청크]")
    print(f"  {original}")
    
    # 보강된 청크
    enriched = create_product_chunk(
        product_id="P001",
        name="무선 블루투스 이어폰",
        description=original,
        reviews=["음질 좋아요", "노이즈 캔슬링 최고"],
        price=89000,
        rating=4.5
    )
    
    print(f"\n[보강된 청크 - 검색용 텍스트]")
    print("-" * 40)
    print(enriched.get_searchable_text())
    print("-" * 40)


if __name__ == "__main__":
    demo_chunk_enrichment()
    demo_ecommerce()
    demo_support()